In [1]:
from pymongo import MongoClient

col_mongo_name = "国家按章分"
# col_mongo_name = "上海按章分"
# col_mongo_name = "中心按章分"
mongo = MongoClient("mongodb://localhost:27017/")
db = mongo["xunfei"]
col_mongo = db[col_mongo_name]


In [2]:
from pymilvus import connections, utility, FieldSchema, CollectionSchema, DataType, Collection
connections.connect("default", host="localhost", port="19530")
col_name = "CountryPolicy"
# col_name = "ShanghaiPolicy"
# col_name = "CentrePolicy"
fields = [
    FieldSchema(name="para_id", dtype=DataType.VARCHAR, is_primary=True, max_length=100),
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=8192),
    FieldSchema(name="dense_vector", dtype=DataType.FLOAT_VECTOR, dim=1024),
    FieldSchema(name="sparse_vector", dtype=DataType.SPARSE_FLOAT_VECTOR)
]

schema = CollectionSchema(fields, description="Policy Paragraph Embeddings")


if utility.has_collection(col_name):
    Collection(col_name).drop()
col = Collection(col_name, schema, consistency_level="Strong")

dense_index = {"index_type": "HNSW", "metric_type": "L2"}
col.create_index("dense_vector", dense_index)
sparse_index = {"index_type": "SPARSE_INVERTED_INDEX", "metric_type": "IP"}
col.create_index("sparse_vector", sparse_index)
# col = Collection(col_name)
col.load()

In [13]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('BAAI/bge-m3',
                      use_fp16=False,
                      pooling_method='cls',
                      devices=['cuda:0'])

def get_embeddings(text):
    embeddings = model.encode(
        text,
        return_dense=True,
        return_sparse=True,
        return_colbert_vecs=False
    )
    return embeddings

Fetching 30 files: 100%|███████████████████████████████████████████████████████████████████████| 30/30 [00:00<?, ?it/s]


In [14]:
from tqdm import tqdm

batch_ids = []
para_ids = []
texts = []
dense_vectors = []
sparse_vectors = []
BATCH_SIZE = 100

for doc in tqdm(col_mongo.find({"vectorized": True}), desc='向量生成中...'):
    para_id = doc['para_id']
    text = doc['text']
    try:
        vector = get_embeddings(text)
        # print(vector)
        # break
        dense_vector = vector["dense_vecs"]
        sparse_vector = vector["lexical_weights"]
        para_ids.append(para_id)
        texts.append(text)
        dense_vectors.append(dense_vector)
        sparse_vectors.append(sparse_vector)
        batch_ids.append(doc['_id'])
        if len(para_ids) > BATCH_SIZE:
            col.insert([para_ids, texts, dense_vectors, sparse_vectors])
            col_mongo.update_many({"_id": {"$in": batch_ids}}, {"$set": {"vectorized": True}})
            batch_ids = []
            para_ids = []
            texts = []
            dense_vectors = []
            sparse_vectors = []

    except Exception as e:
        print(f"向量化失败：{para_id}, {type(e).__name__}: {e}")

if para_ids:
    col.insert([para_ids, texts, dense_vectors, sparse_vectors])
    col_mongo.update_many({"_id": {"$in": batch_ids}}, {"$set": {"vectorized": True}})
print("生成已完成。")

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
向量生成中...: 6277it [04:10, 25.10it/s]

生成已完成。


In [5]:
from pymilvus import AnnSearchRequest, WeightedRanker

def dense_search(col, query_dense_embedding, limit=5):
    search_params = {"metric_type": "L2", "params": {"level":1}}
    res = col.search(
        [query_dense_embedding],
        anns_field="dense_vector",
        limit=limit,
        output_fields=["text"],
        param=search_params,
    )[0]
    return [hit.get("text") for hit in res]


def sparse_search(col, query_sparse_embedding, limit=5):
    search_params = {
        "metric_type": "IP",
        "params": {},
    }
    res = col.search(
        [query_sparse_embedding],
        anns_field="sparse_vector",
        limit=limit,
        output_fields=["text"],
        param=search_params,
    )[0]
    return [hit.get("text") for hit in res]


def hybrid_search(
    col,
    query_dense_embedding,
    query_sparse_embedding,
    sparse_weight=1.0,
    dense_weight=1.0,
    limit=20,
):
    dense_search_params = {"metric_type": "L2", "params": {"level":1}}
    dense_req = AnnSearchRequest(
        [query_dense_embedding], "dense_vector", dense_search_params, limit=limit
    )
    sparse_search_params = {"metric_type": "IP", "params": {}}
    sparse_req = AnnSearchRequest(
        [query_sparse_embedding], "sparse_vector", sparse_search_params, limit=limit
    )
    rerank = WeightedRanker(sparse_weight, dense_weight)
    res = col.hybrid_search(
        [sparse_req, dense_req], rerank=rerank, limit=limit, output_fields=["text"]
    )[0]
    return [hit.get("text") for hit in res]

In [6]:
import jieba

def doc_text_formatting(query, docs):
    query_words = list(set(jieba.lcut(query)))  # 中文分词
    formatted_texts = []

    for doc in docs:
        highlighted = doc
        for word in query_words:
            if not word.strip():
                continue
            highlighted = re.sub(
                re.escape(word),
                f"<span style='color:red'>{word}</span>",
                highlighted
            )
        formatted_texts.append(highlighted)
    return formatted_texts


In [18]:
query = input("Enter your search query: ")
query_embeddings = get_embeddings([query])

Enter your search query:  评估专业人员在何种情形下不得从事评估业务？境外国有产权发生变动时，是否需要向国资委申请产权登记？


In [19]:
dense_results = dense_search(col, query_embeddings["dense_vecs"][0])
sparse_results = sparse_search(col, query_embeddings["lexical_weights"][0])
hybrid_results = hybrid_search(
    col,
    query_embeddings["dense_vecs"][0],
    query_embeddings["lexical_weights"][0],
    sparse_weight=0.7,
    dense_weight=1.0,
)

In [20]:
import re
from IPython.display import Markdown, display

# Dense 搜索结果
display(Markdown("### 🔍 **Dense Search Results:**"))
formatted_results = doc_text_formatting(query, dense_results)
for result in formatted_results:
    display(Markdown(result))

# Sparse 搜索结果
display(Markdown("### 🌿 **Sparse Search Results:**"))
formatted_results = doc_text_formatting(query, sparse_results)
for result in formatted_results:
    display(Markdown(result))

# Hybrid 搜索结果
display(Markdown("### 🔄 **Hybrid Search Results:**"))
formatted_results = doc_text_formatting(query, hybrid_results)
for result in formatted_results:
    display(Markdown(result))


### 🔍 **Dense Search Results:**

第九条 第九条中央企业及其各级子企业以其拥有的境内<span style='color:red'>国有<span style='color:red'>产权</span></span><span style='color:red'>向</span><span style='color:red'>境外</span>企业注资或者转让<span style='color:red'>，</span>或者以其拥有的<span style='color:red'>境外</span><span style='color:red'>国有<span style='color:red'>产权</span></span><span style='color:red'>向</span>境内企业注资或者转让<span style='color:red'>，</span>应当依照《企业国有资产<span style='color:red'>评估</span>管理暂行办法》（<span style='color:red'>国资委</span>令第12号）等相关规定<span style='color:red'>，</span>聘请具有相应资质的境内<span style='color:red'>评估</span>机构对标的物进行<span style='color:red'>评估</span><span style='color:red'>，</span>并办理<span style='color:red'>评估</span>备案或者核准。

第十条 第十条中央企业及其各级子企业独资或者控股的<span style='color:red'>境外</span>企业<span style='color:red'>在</span><span style='color:red'>境外</span><span style='color:red'>发生</span>转让或者受让<span style='color:red'>产权</span>、以非货币资产出资、非上市公司国有股东股权比例<span style='color:red'>变动</span>、合并分立、解散清算等经济行为<span style='color:red'>时</span><span style='color:red'>，</span>应当聘请具有相应资质、<span style='color:red'>专业</span>经验和良好信誉的<span style='color:red'>专业</span>机构对标的物进行<span style='color:red'>评估</span>或者估值<span style='color:red'>，</span><span style='color:red'>评估</span>项目或者估值情况应当由中央企业备案；涉及中央企业重要子企业由国有独资转为绝对控股、绝对控股转为相对控股或者失去控股地位等经济行为的<span style='color:red'>，</span><span style='color:red'>评估</span>项目或者估值情况应当报<span style='color:red'>国资委</span>备案或者核准。中央企业及其各级子企业独资或者控股的<span style='color:red'>境外</span>企业<span style='color:red'>在</span>进行与<span style='color:red'>评估</span>或者估值相应的经济行为<span style='color:red'>时</span><span style='color:red'>，</span>其交易对价应当以经备案的<span style='color:red'>评估</span>或者估值结果为基准。

第二十九条 第二十九条备案管理单位审核<span style='color:red'>境外</span><span style='color:red'>评估</span>或估值报告<span style='color:red'>，</span>应当关注<span style='color:red'>评估</span>或估值机构的选聘<span style='color:red'>是否</span>符合《中央企业<span style='color:red'>境外</span><span style='color:red'>国有<span style='color:red'>产权</span></span>管理暂行办法》（<span style='color:red'>国资委</span>令第27号）和《关于加强中央企业<span style='color:red'>境外</span><span style='color:red'>国有<span style='color:red'>产权</span></span>管理有关工作的通知》（国资发<span style='color:red'>产权</span>〔2011〕144号）等规定。

第八条 第八条中央企业及其各级子企业<span style='color:red'>发生</span>以<span style='color:red'>下</span>事项<span style='color:red'>时</span><span style='color:red'>，</span>应当由中央企业统一<span style='color:red'>向</span><span style='color:red'>国资委</span>申办<span style='color:red'>产权</span><span style='color:red'>登记</span>：（一）以投资、分立、合并等方式新设<span style='color:red'>境外</span>企业<span style='color:red'>，</span>或者以收购、投资入股等方式首次取得<span style='color:red'>境外</span>企业<span style='color:red'>产权</span>的。（二）<span style='color:red'>境外</span>企业名称、注册地、注册资本、主营<span style='color:red'>业务</span>范围等企业基本信息<span style='color:red'>发生</span>改变<span style='color:red'>，</span>或者因企业出资人、出资额、出资比例等变化导致<span style='color:red'>境外</span>企业<span style='color:red'>产权</span>状况<span style='color:red'>发生</span>改变的。（三）<span style='color:red'>境外</span>企业解散、破产<span style='color:red'>，</span>或者因<span style='color:red'>产权</span>转让、减资等原因不再保留<span style='color:red'>国有<span style='color:red'>产权</span></span>的。（四）其他<span style='color:red'>需要</span>办理<span style='color:red'>产权</span><span style='color:red'>登记</span>的<span style='color:red'>情形</span>。

...

第四条 第四条占有单位有<span style='color:red'>下</span>列行为之一的<span style='color:red'>，</span>可以不进行资产<span style='color:red'>评估</span>：（一）经各级人民政府及其授权部门批准<span style='color:red'>，</span>对整体企业或者部分资产实施无偿划转；（二）国有独资企业、行政事业单位<span style='color:red'>下</span>属的独资企业（事业单位）之间的合并、资产（<span style='color:red'>产权</span>）划转、置换和转让。

第三十六条 第三十六条<span style='color:red'>境外</span>国有资产的<span style='color:red'>评估</span><span style='color:red'>，</span>不适用本办法。

第四十七条 第四十七条国有独资企业、国有独资公司和国有资本控股公司合并、分立、改制<span style='color:red'>，</span>转让重大财产<span style='color:red'>，</span>以非货币财产对外投资<span style='color:red'>，</span>清算或者有法律、行政法规以及企业章程规定应当进行资产<span style='color:red'>评估</span>的其他<span style='color:red'>情形</span>的<span style='color:red'>，</span>应当按照规定对有关资产进行<span style='color:red'>评估</span>。

第二十九条 第二十九条受托资产<span style='color:red'>评估</span>机构<span style='color:red'>在</span>资产<span style='color:red'>评估</span>过程中违规执业的<span style='color:red'>，</span>由国有资产监督管理机构将有关情况通报其行业主管部门<span style='color:red'>，</span>建议给予相应处罚；情节严重的<span style='color:red'>，</span>可要求企业<span style='color:red'>不得</span>再委托该中介机构及其当事人进行国有资产<span style='color:red'>评估</span><span style='color:red'>业务</span>；涉嫌犯罪的<span style='color:red'>，</span>依法移送司法机关处理。

第四十八条 第四十八条国有独资企业、国有独资公司和国有资本控股公司应当委托依法设立的符合条件的资产<span style='color:red'>评估</span>机构进行资产<span style='color:red'>评估</span>；涉及应当报经履行出资人职责的机构决定的事项的<span style='color:red'>，</span>应当将委托资产<span style='color:red'>评估</span>机构的情况<span style='color:red'>向</span>履行出资人职责的机构报告。

In [10]:
from FlagEmbedding import BGEM3FlagModel
sentences_1 = ["你是谁？", "我已有MongoDB文档，之后该怎么做？"]
sentences_2 = ["需要设计一个具体的分割与存储方案，结合你已有的 MongoDB 文档结构", "可以根据你的MongoDB文档向量化"]
model = BGEM3FlagModel('BAAI/bge-m3',
                       use_fp16=True,
                       pooling_method='cls',
                       devices=['cuda:0']) # Setting use_fp16 to True speeds up computation with a slight performance degradation
embeddings_1 = model.encode(
    sentences_1,
    return_dense=True,
    return_sparse=True,
    return_colbert_vecs=False,
)
embeddings_2 = model.encode(
    sentences_2,
    return_dense=True,
    return_sparse=True,
    return_colbert_vecs=False,
)
dense_similarity = embeddings_1["dense_vecs"] @ embeddings_2["dense_vecs"].T
print('dense similarity:', dense_similarity)
sparse_similarity = model.compute_lexical_matching_score(
    embeddings_1["lexical_weights"],
    embeddings_2["lexical_weights"],
)
print('sparse similarity:', sparse_similarity)

queries = ['query_1', 'query_2']
passages = ["样例文档-1", "样例文档-2"]
q_embeddings = model.encode_queries(
    queries,
    return_dense=True,
    return_sparse=True,
    return_colbert_vecs=False,
)
p_embeddings = model.encode_corpus(
    passages,
    return_dense=True,
    return_sparse=True,
    return_colbert_vecs=False,
)
dense_scores = q_embeddings["dense_vecs"] @ p_embeddings["dense_vecs"].T
print('dense scores:', dense_scores)
sparse_scores = model.compute_lexical_matching_score(
    q_embeddings["lexical_weights"],
    p_embeddings["lexical_weights"],
)
print('sparse similarity:', sparse_scores)

Fetching 30 files: 100%|███████████████████████████████████████████████████████████████████████| 30/30 [00:00<?, ?it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


dense similarity: [[0.279  0.3164]
 [0.7773 0.748 ]]
sparse similarity: [[0.         0.        ]
 [0.22802734 0.2565918 ]]
dense scores: [[0.558  0.4692]
 [0.475  0.573 ]]
sparse similarity: [[0 0]
 [0 0]]
